In [27]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 1: The Tyranny of the Rocket Equation ---</h2><p>Santa has become stranded at the edge of the Solar System while delivering presents to other planets! To accurately calculate his position in space, safely align his warp drive, and return to Earth in time to save Christmas, he needs you to bring him <span title="If only you had time to grab an astrolabe.">measurements</span> from <em class="star">fifty stars</em>.</p>
<p>Collect stars by solving puzzles.  Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first.  Each puzzle grants <em class="star">one star</em>. Good luck!</p>
<p>The Elves quickly load you into a spacecraft and prepare to launch.</p>
<p>At the first Go / No Go poll, every Elf is Go until the Fuel Counter-Upper.  They haven't determined the amount of fuel required yet.</p>
<p>Fuel required to launch a given <em>module</em> is based on its <em>mass</em>.  Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.</p>
<p>For example:</p>
<ul>
<li>For a mass of <code>12</code>, divide by 3 and round down to get <code>4</code>, then subtract 2 to get <code>2</code>.</li>
<li>For a mass of <code>14</code>, dividing by 3 and rounding down still yields <code>4</code>, so the fuel required is also <code>2</code>.</li>
<li>For a mass of <code>1969</code>, the fuel required is <code>654</code>.</li>
<li>For a mass of <code>100756</code>, the fuel required is <code>33583</code>.</li>
</ul>
<p>The Fuel Counter-Upper needs to know the total fuel requirement.  To find it, individually calculate the fuel needed for the mass of each module (your puzzle input), then add together all the fuel values.</p>
<p><em>What is the sum of the fuel requirements</em> for all of the modules on your spacecraft?</p>
</article>


In [28]:
tests = [
    {
        "name": "Example 1",
        "mass": 12,
        "expected": 2,
    },
    {
        "name": "Example 2",
        "mass": 14,
        "expected": 2,
    },
    {
        "name": "Example 3",
        "mass": 1969,
        "expected": 654,
    },
    {
        "name": "Example 4",
        "mass": 100756,
        "expected": 33583,
    },
]


def fuell(mass: int) -> int:
    """
    Fuel required to launch a given module is based on its mass.
    Specifically, to find the fuel required for a module, take its mass,
    divide by three, round down, and subtract 2.
    """
    return mass // 3 - 2


run_tests_params(fuell, tests)

example = """
12
14
1969
100756
"""


def sum_fuell(s: str) -> int:
    return sum(fuell(int(l.strip())) for l in s.strip().splitlines())


print()
print(f"Example: {sum_fuell(example)} should be {2 + 2 + 654 + 33583}")


Test Example 1 passed, for fuell.
Test Example 2 passed, for fuell.
Test Example 3 passed, for fuell.
Test Example 4 passed, for fuell.
Success

Example: 34241 should be 34241


In [29]:
with open("../input/day1.txt") as f:
    puzzle = f.read()

print(f"Part I: {sum_fuell(puzzle)}")

Part I: 3270338


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3270338</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>During the second Go / No Go poll, the Elf in charge of the Rocket Equation Double-Checker stops the launch sequence.  Apparently, you forgot to include additional fuel for the fuel you just added.</p>
<p>Fuel itself requires fuel just like a module - take its mass, divide by three, round down, and subtract 2.  However, that fuel <em>also</em> requires fuel, and <em>that</em> fuel requires fuel, and so on.  Any mass that would require <em>negative fuel</em> should instead be treated as if it requires <em>zero fuel</em>; the remaining mass, if any, is instead handled by <em>wishing really hard</em>, which has no mass and is outside the scope of this calculation.</p>
<p>So, for each module mass, calculate its fuel and add it to the total.  Then, treat the fuel amount you just calculated as the input mass and repeat the process, continuing until a fuel requirement is zero or negative. For example:</p>
<ul>
<li>A module of mass <code>14</code> requires <code>2</code> fuel.  This fuel requires no further fuel (2 divided by 3 and rounded down is <code>0</code>, which would call for a negative fuel), so the total fuel required is still just <code>2</code>.</li>
<li>At first, a module of mass <code>1969</code> requires <code>654</code> fuel.  Then, this fuel requires <code>216</code> more fuel (<code>654 / 3 - 2</code>).  <code>216</code> then requires <code>70</code> more fuel, which requires <code>21</code> fuel, which requires <code>5</code> fuel, which requires no further fuel.  So, the total fuel required for a module of mass <code>1969</code> is <code>654 + 216 + 70 + 21 + 5 = 966</code>.</li>
<li>The fuel required by a module of mass <code>100756</code> and its fuel is: <code>33583 + 11192 + 3728 + 1240 + 411 + 135 + 43 + 12 + 2 = 50346</code>.</li>
</ul>
<p><em>What is the sum of the fuel requirements</em> for all of the modules on your spacecraft when also taking into account the mass of the added fuel? (Calculate the fuel requirements for each module separately, then add them all up at the end.)</p>
</article>

</main>


In [30]:
tests = [
    {
        "name": "Example 1",
        "mass": 12,
        "expected": 2,
    },
    {
        "name": "Example 2",
        "mass": 14,
        "expected": 2,
    },
    {
        "name": "Example 3",
        "mass": 1969,
        "expected": 966,
    },
    {
        "name": "Example 4",
        "mass": 100756,
        "expected": 50346,
    },
]


def fuell_total(mass: int) -> int:
    total_fuell = 0
    mass = fuell(mass)

    while mass > 0:
        total_fuell += mass
        mass = fuell(mass)

    return total_fuell


run_tests_params(fuell_total, tests)

example = """
12
14
1969
100756
"""


def sum_total_fuell(s: str) -> int:
    return sum(fuell_total(int(l.strip())) for l in s.strip().splitlines())


print()
print(f"Example: {sum_total_fuell(example)} should be {2 + 2 + 966 + 50346}")


Test Example 1 passed, for fuell_total.
Test Example 2 passed, for fuell_total.
Test Example 3 passed, for fuell_total.
Test Example 4 passed, for fuell_total.
Success

Example: 51316 should be 51316


In [31]:
print(f"Part II: {sum_total_fuell(puzzle)}")

Part II: 4902650


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4902650</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
